# Brief of the notebook

 The notebook consists of code to all the experiments performed, with all the CNN architectures mentioned in the report getting used. The preprocessing code of both the images as well as the annotations along with the feature generator input to the LSTM and the caption generator code. 

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
import matplotlib as matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import string
from PIL import Image
import glob
import keras
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import os
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from time import time
import pickle
from pickle import dump, load
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras import Input, layers
from keras.preprocessing import sequence
from keras import optimizers
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.utils.vis_utils import plot_model
import pydotplus
import pydot
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from datetime import datetime
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import nasnet
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Setting up the captions
file = r'flickr8kcaptions.txt'
file = open(file, 'r')
doc = file.read()

In [ ]:
#Setting up the ids
train_dataid = 'train_dataid.txt'
file2 = open(train_dataid, 'r')
doc2 = file2.read()

In [ ]:
# Creating a dictionary where key is the image ID and its values is an array of its five captions
description = dict()
desc = []   
imageID =[]
lines = doc.split('\n')
#lines = lines[1:]

for line in lines:
    if line:
        token = line.split('\t')
        imageId,desc = token[0],token[1]
        imageId = imageId.split('.')[0]
        if imageId not in description:
            description[imageId] = list()
            description[imageId].append(desc)
        else:
            description[imageId].append(desc)       


In [ ]:
description['1000268201_693b08cb0e']

In [ ]:
#cleaning the annotations data for whole dataset.
table = str.maketrans('','', string.punctuation)
for key, desc_list in description.items():
    for i in range(len(desc_list)):
        desc = desc_list[i]
        desc = desc.split()
        desc = [word.lower() for word in desc ]
        desc = [w.translate(table) for w in desc]
        desc = [word for word in desc if len(word)>1]
        desc = [word for word in desc if word.isalpha()]
        desc_list[i] = ' '.join(desc)

In [ ]:
#Taking out captions for training set
content = doc2.split('\n')
tdict = dict()
train_imageid = []
for ID in content:
    if ID:
        train_imageid.append(ID.split('.')[0])
for img in train_imageid:
    if img in description:
        tdict[img] = []
        for desc in description[img]:
            tdict[img].append('startseq ' + desc + ' endseq')

        

        

tdict['1003163366_44323f5815']  

In [ ]:
vocabulary = set()
for key in description.keys():
    [vocabulary.update(d.split()) for d in description[key]]
print('original vocabulary size is %d' %len(vocabulary))
print(vocabulary)

In [ ]:
# taking out vocabulary words that are very less frequent

all_captions = []

for key, val in description.items():
    for captions in val:
        all_captions.append(captions)


threshold = 10
word_count = {}
nsents = 0

for sent in all_captions:
    nsents  +=1
    for w in sent.split(' '):
        word_count[w] = word_count.get(w, 0) + 1

vocab  = [word for word in word_count if word_count[word] >= threshold]
len(vocab)


In [ ]:
#Loads train images IDs.


train_id =[]
trainid_file = 'train_dataid.txt'
doc = open(trainid_file, 'r')
img_id = doc.read()
img_id = img_id.split('\n')

for Id in img_id:
    if Id:
        sep = Id.split('.')[0]
        train_id.append(sep)
print('dataset: %d' %len(train_id))

In [ ]:
valdict={}
val_id = []
val_file = 'val_dataid.txt'
val_doc = open(val_file,'r')
val_data = val_doc.read()
val_data = val_data.split('\n')

for Id in val_data:
    if Id:
        sep = Id.split('.')[0]
        val_id.append(sep)
for img in val_id:
    if img in description:
        valdict[img] = []
        for desc in description[img]:
            valdict[img].append('startseq ' + desc + ' endseq')
print('dataset: %d' %len(val_id))

In [ ]:
# Loading images

images = 'Images/Images/'

img = glob.glob(images + '*.jpg')
print(img)


In [ ]:
# Images to be used in training the models

train_images = set(open('train_dataid.txt', 'r').read().strip().split('\n'))

train_img = []

for i in img:
    j = i.split('/')[-1]
    if j in train_images:
        train_img.append(i)

print(train_img)

In [ ]:
#Images of validation data

val_path = 'val_dataid.txt'

val_images = set(open(val_path, 'r').read().strip().split('\n'))
val_img = []

for i in img:
    j = i.split('/')[-1]
    if j in val_images:
        val_img.append(i)
            

In [ ]:
# Images of the test data

test_path = 'test_dataid.txt'


test_images = set(open(test_path, 'r').read().strip().split('\n'))
test_img = []

for i in img:
    j = i.split('/')[-1]
    if j in test_images:
        test_img.append(i)


In [ ]:
def preprocess(img):
    all_images = image.load_img(img, target_size=(299,299))
    x_axis = image.img_to_array(all_images)
    print(x_axis.shape)
    x_axis = np.expand_dims(x_axis, axis = 0)
    print(x_axis.shape)
    x_axis = preprocess_input(x_axis)
    return x_axis

In [ ]:
for f in os.listdir('Images/Images'):
    all_images = image.load_img(f'Images/Images/{f}', target_size=(299,299))
    x_axis = image.img_to_array(all_images)
    print(x_axis.shape)
    x_axis = np.expand_dims(x_axis, axis = 0)
    print(x_axis.shape)
    x_axis = preprocess_input(x_axis)


In [ ]:
#Preprocess all images
trial_img = image.load_img('pic_screen.png', target_size = (299, 299))
x = image.img_to_array(trial_img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)


INCEPTION V3

In [ ]:
# Instancing the inceptionv3 model and using the trained model on imagenet dataset

model = InceptionV3(weights = 'imagenet' )

In [ ]:
len(model.layers)

In [ ]:
#Removing the softmax layer from the neural network

cnn = Model(model.input, model.layers[-2].output)


In [ ]:
def plot_filter(layer, x,y):
    filters = layer.get_weights()
    fig = plt.figure()
    for i in range(len(filters)):
        ax = fig.add_subplot(y,x, i+1)
        ax.matshow(filters[i][0], cmap = matplotlib.cm.binary)
        plt.xticks(np.array([]))
        plt.yticks(np.array([]))
    plt.tight_layout()
    return plt

In [ ]:
# To encode an image to a feature vector of (2048,)

def vector(img):
    img = preprocess(img)
    featvec = cnn.predict(img)
    featvec = np.reshape(featvec, featvec.shape[1])
    return featvec
  

In [ ]:
# encoding all images

check = time()
enc_train = {}

for img in train_img:
    enc_train[img[len(images):]] = vector(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_trainimg.pkl', 'wb') as enc_pkl:
    pickle.dump(enc_train, enc_pkl)

In [ ]:
# encoding all images

check = time()
enc_val = {}

for img in val_img:
    enc_val[img[len(images):]] = vector(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
#with open('enc_valimg.pkl', 'wb') as enc_valpkl:
#    pickle.dump(enc_val, enc_valpkl)

In [ ]:
# Encoding all test images
check = time()
enc_test = dict()

for img in test_img:
    enc_test[img[len(images):]] = vector(img)
print('time taken in sec', time()-check)

In [ ]:
trial_vector = vector('pic_screen.png')

In [ ]:
#Visualising CNN layers

layer_names = []

for layer in cnn.layers[:8]:
    layer_names.append(layer.name)
    

In [ ]:
#Saving test feature

with open('enc_testing.pkl', 'wb') as enc_pickle:
    pickle.dump(enc_test, enc_pickle)

In [ ]:
train_features = load(open('enc_trainimg.pkl','rb'))


In [ ]:
val_features = load(open('enc_valimg.pkl', 'rb'))

In [ ]:
# Gathering all the train captions

all_traincap =[]

for key, val in tdict.items():
    for cap in val:
        all_traincap.append(cap)

In [ ]:
wrd = [i.split() for i in all_traincap]

In [ ]:
unique = []

for i in wrd:
    unique.extend(i)

In [ ]:
unique = list(set(unique))

In [ ]:
len(unique)

In [ ]:
with open('unique.pkl', 'wb') as pickle_d:
    pickle.dump(unique, pickle_d)

In [ ]:
unique = pickle.load(open('unique.pkl', 'rb'))

In [ ]:
wordtoix2 = {val:index for index, val in enumerate(unique)}

In [ ]:
wordtoix2['startseq']

In [ ]:
ixtoword2 = {index:val for index, val in enumerate(unique)}

In [ ]:
ixtoword2[1847]

In [ ]:
#Creating word and its index

wordtoix = {}
ixtoword ={}

ix = 1
for word in vocab:
    wordtoix[word] = ix
    ixtoword[ix] = word
    ix +=1

In [ ]:
vocab_size = len(ixtoword) + 1

In [ ]:
# Taking out all the captions

def app_cap(desc):
    tot_cap =[]
    for key, val in description.items():
        for cap in val:
            tot_cap.append(cap)
    return tot_cap

# Finding max length of captions


def max_length(captions):
    sen = app_cap(captions)
    return max(len(line.split()) for line in sen)

In [ ]:
max_length = max_length(tdict)
print('The max length of caption is: %d ' % max_length)

In [ ]:
def data_generator(description, pics, wordtoix, max_length, num_photos_per_batch):
    X1, X2, Y = list(), list(), list()
    n = 0
    while 1:
        for key, list_desc in description.items():
            n+=1
            photos = pics[key + '.jpg']
            for desc in list_desc:
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                for i in range(1, len(seq)):
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen = max_length)[0]
                    out_seq = to_categorical([out_seq], num_classes= vocab_size)[0]
                    X1.append(photos)
                    X2.append(in_seq)
                    Y.append(out_seq)
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(Y)]
                X1, X2, Y = list(), list(), list()
                n=0
    

In [ ]:
generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)

In [ ]:
#Using a pretrained Glove model 

glove_dir = 'Glove200'

embedding_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding = 'utf-8') 

for line in f:
    values = line.split()
    word = values[0]
    coef = np.asarray(values[1:], dtype = 'float32')
    embedding_index[word] = coef
f.close()

print(len(embedding_index))

In [ ]:
emb_dim = 200

embedding_matrix = np.zeros((vocab_size, emb_dim))

for word, i in wordtoix.items():
    emb_vector = embedding_index.get(word)
    if emb_vector is not None:
        embedding_matrix[i] = emb_vector

In [ ]:
# LSTM 1

inputs1 = Input(shape = (2048,))
fe1 = layers.Dropout(0.5)(inputs1)
fe2 = Dense(512, activation = 'relu')(fe1)


inputs2 = Input(shape = (max_length,))
se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(512)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(512, activation = 'relu')(decoder1)
outputs = Dense(vocab_size, activation = 'softmax')(decoder2)
model0 = Model(inputs= [inputs1, inputs2], output= outputs)
#plot_model(model0, to_file='plot_model/model', )

In [ ]:
# LSTM 2

inputs1 = Input(shape = (2048,))
fe1 = Dense(512, activation = 'relu')(inputs1)
fe2 = layers.Dropout(0.5)(fe1)

inputs2 = Input(shape = (max_length,))
se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inputs2)
se2 = LSTM(512)(se1)
decoder1 = add([fe2, se2])
decoder2 = Dense(512, activation = 'relu')(decoder1)
se3 = Dropout(0.5)(decoder2)
outputs = Dense(vocab_size, activation = 'softmax')(se3)
model1 = Model(inputs= [inputs1, inputs2], output= outputs)
#pydot = keras.utils.vis_utils.pydot
#plot_model(model0, to_file='plot_model/model.png', show_shapes=True)

In [ ]:
# LSTM #3

inputs1 = Input(shape = (2048,))
fe1 = Dense(512, activation = 'relu')(inputs1)
fe2 = layers.Dropout(0.5)(fe1)

inputs2 = Input(shape = (max_length,))
se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inputs2)
se2 = LSTM(512)(se1)
decoder1 = add([fe2, se2])
se3 = Dropout(0.5)(decoder1)
decoder2 = Dense(512, activation = 'relu')(se3)

outputs = Dense(vocab_size, activation = 'softmax')(decoder2)
model2 = Model(inputs= [inputs1, inputs2], output= outputs)

#plot_model(model0, to_file='plot_model/model.png', show_shapes=True)

In [ ]:
# LSTM 4

inputs1 = Input(shape = (2048,))
fe1 = Dense(512, activation = 'relu')(inputs1)
fe2 = layers.Dropout(0.5)(fe1)

inputs2 = Input(shape = (max_length,))
se1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inputs2)
se2 = LSTM(512)(se1)
decoder1 = add([fe2, se2])
se3 = Dropout(0.5)(decoder1)

outputs = Dense(vocab_size, activation = 'softmax')(se3)
model3 = Model(inputs= [inputs1, inputs2], output= outputs)

#plot_model(model0, to_file='plot_model/model.png', show_shapes=True)

In [ ]:
vocab_size

In [ ]:
#LSTM 1
inputs1 = layers.Input(shape = (2048,))
fe1 = layers.Dropout(0.5)(inputs1)
fe2 = layers.Dense(1024, activation = 'relu')(fe1)

inputs2 = layers.Input(shape = (max_length,))
se1 = layers.Embedding(vocab_size, emb_dim)(inputs2)
se2 = layers.Dropout(0.5)(se1)
se3 = CuDNNLSTM(1024)(se2)
decoder1 = layers.add([fe2, se3])
decoder2 = layers.Dense(1024, activation = 'relu')(decoder1)
outputs = layers.Dense(vocab_size, activation = 'softmax')(decoder2)
model0 = Model([inputs1, inputs2], outputs)


In [ ]:
model0.layers[2].set_weights([embedding_matrix])
model0.layers[2].trainable = False

In [ ]:
model1.layers[3].set_weights([embedding_matrix])
model1.layers[3].trainable = False

In [ ]:
model2.layers[3].set_weights([embedding_matrix])
model2.layers[3].trainable = False

In [ ]:
model3.layers[3].set_weights([embedding_matrix])
model3.layers[3].trainable = False

In [ ]:
model0.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
model1.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
model3.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
epochs = 20
number_photo_per_batch = 32
steps = len(tdict)// number_photo_per_batch

In [ ]:
#Epochs with batch size 32 and dense units as 256
loss0_batch32=[]
accuracy0_batch32 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_batch32.append(history.history['val_loss'])
    accuracy0_batch32.append(history.history['val_accuracy'])
    model0.save('model_weights/model' + str(j) +'.h5' )

In [ ]:
#Epochs with batch size 64

loss1_batch64=[]
accuracy1_batch64 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1,  validation_data = generator2, validation_steps = steps )
    loss1_batch64.append(history.history['val_loss'])
    accuracy1_batch64.append(history.history['val_accuracy'])
    model0.save('model_weights_batch64/model' + str(j) +'.h5' )

In [ ]:
# Epochs with batch size 128
loss2_batch128=[]
accuracy2_batch128 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1,  validation_data = generator2, validation_steps = steps )
    loss2_batch128.append(history.history['val_loss'])
    accuracy2_batch128.append(history.history['val_accuracy'])
    model0.save('model_weights_batch128/model' + str(j) +'.h5' )

In [ ]:
# Epochs with batch size 256
loss3_batch256=[]
accuracy3_batch256 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1,  validation_data = generator2, validation_steps = steps )
    loss3_batch256.append(history.history['val_loss'])
    accuracy3_batch256.append(history.history['val_accuracy'])
    model0.save('model_weights_batch256/model' + str(j) +'.h5' )

# Experiments with Hidden units (LSTM)

In [ ]:
#Plots the graph of val accuracy and loss with hidden units
plt.figure(figsize = (25,15))
plt.plot(loss1, linewidth = 5)
plt.plot(loss0, linewidth = 5)
plt.plot(loss2, linewidth = 5)
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Dense-256', 'Dense-512', 'Dense-1024'])
plt.show

In [ ]:
plt.figure(figsize = (25,15))
plt.plot(accuracy1, linewidth = 5)
plt.plot(accuracy0, linewidth = 5)
plt.plot(accuracy2, linewidth = 5)
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Dense-256', 'Dense-512', 'Dense-1024'])
plt.show

# Experiments with batch size

In [ ]:
matplotlib.rcParams.update({'font.size': 25})
plt.figure(figsize=(25,15))
plt.plot(loss0_batch32, linewidth=5)
plt.plot(loss1_batch64, linewidth = 5)
plt.plot(loss2_batch128, linewidth = 5, c='grey')
plt.plot(loss3_batch256, linewidth=5)
plt.ylabel('Validation Loss ', fontsize=25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Batch-32', 'Batch-64', 'Batch-128', 'Batch-256'], loc='upper right', prop={'size':22} )
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 25})
plt.figure(figsize=(25,15))
plt.plot(accuracy0_batch32, linewidth=5)
plt.plot(accuracy1_batch64, linewidth=5)
plt.plot(accuracy2_batch128, linewidth=5, c = 'grey')
plt.plot(accuracy3_batch256, linewidth=5)

plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Batch-32', 'Batch-64','Batch-128','Batch-256'], loc='upper left', prop = {'size': 22} )
plt.show()

#  Experiments with hidden units

In [ ]:
#Epochs with hidden unit 256

loss0_hidden = loss0_batch
accuracy0_hidden = accuracy0_batch

In [ ]:
#Epochs with hidden unit 512 (LSTM) with batch size kept 32 constant
loss0_hidden256=[]
accuracy0_hidden256 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_hidden256.append(history.history['val_loss'])
    accuracy0_hidden256.append(history.history['val_accuracy'])
    model0.save('model_hidden256/model' + str(j) +'.h5' )

In [ ]:
#Epochs with hidden unit 512 (LSTM) with batch size kept 32 constant
loss1_hidden512=[]
accuracy1_hidden512 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss1_hidden512.append(history.history['val_loss'])
    accuracy1_hidden512.append(history.history['val_accuracy'])
    model0.save('model_hidden512/model' + str(j) +'.h5' )

In [ ]:
#Epochs with hidden unit 1024 (LSTM) with batch size kept 32 constant
loss2_hidden1024=[]
accuracy2_hidden1024 = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss2_hidden1024.append(history.history['val_loss'])
    accuracy2_hidden1024.append(history.history['val_accuracy'])
    model0.save('model_hidden1024/model' + str(j) +'.h5' )

In [ ]:
#Visualisation
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(loss0_hidden256, linewidth = 5)
plt.plot(loss1_hidden512, linewidth = 5)
plt.plot(loss2_hidden1024, linewidth = 5)
plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Dense-256', 'Dense-512', 'Dense-1024'], prop={'size':22})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(accuracy0_hidden256, linewidth = 5)
plt.plot(accuracy1_hidden512, linewidth = 5)
plt.plot(accuracy2_hidden1024, linewidth = 5)
plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Dense-256', 'Dense-512', 'Dense-1024'], prop={'size':22})
plt.show

# Experiments with Dropout layers

In [ ]:
#Epochs with dropout layer changed from 2nd to 3rd and 6th to 9th with no dense layer removed.
loss0_drop=[]
accuracy0_drop = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model0.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_drop.append(history.history['val_loss'])
    accuracy0_drop.append(history.history['val_accuracy'])
    model0.save('model_weights_drop0/model' + str(j) +'.h5' )

In [ ]:
#Epochs with dropout layer changed from 2nd to 3rd and 6th to 9th with no dense layer removed.
loss1_drop=[]
accuracy1_drop = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model1.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss1_drop.append(history.history['val_loss'])
    accuracy1_drop.append(history.history['val_accuracy'])
    model1.save('model_weights_drop1/model' + str(j) +'.h5' )

In [ ]:
#Epochs with dropout layer changed from 2nd to 3rd and 6th to 9th with no dense layer removed.
loss2_drop=[]
accuracy2_drop = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model2.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss2_drop.append(history.history['val_loss'])
    accuracy2_drop.append(history.history['val_accuracy'])
    model2.save('model_weights_drop2/model' + str(j) +'.h5' )

In [ ]:
#Epochs with dropout layer changed from 2nd to 3rd and 6th to 9th with no dense layer removed.
loss3_drop=[]
accuracy3_drop = []
for j in range(epochs):
    generator = data_generator(tdict, train_features, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features, wordtoix, max_length, number_photo_per_batch)
    history=model3.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss3_drop.append(history.history['val_loss'])
    accuracy3_drop.append(history.history['val_accuracy'])
    model3.save('model_weights_drop3/model' + str(j) +'.h5' )

In [ ]:
#Visualising experiments with dropout layers

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(accuracy0_drop, linewidth = 5)
plt.plot(accuracy1_drop, linewidth = 5)
plt.plot(accuracy2_drop, linewidth = 5)
plt.plot(accuracy3_drop, linewidth = 5)
plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Dropout layer after Input-1 and Enbedding Layer', 'Dropout layer after first and second Dense layers ', \
            'Dropout layer after first Dense layer and after addition layer',\
            'Dropout layer after first Dense layer and after addition layer removing second last dense layer'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(loss0_drop, linewidth = 5)
plt.plot(loss1_drop, linewidth = 5)
plt.plot(loss2_drop, linewidth = 5)
plt.plot(loss3_drop, linewidth = 5)
plt.plot(loss3_drop, linewidth = 5)
plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Dropout layer after Input-1 and Enbedding Layer', 'Dropout layer after first and second Dense layers ', \
            'Dropout layer after first Dense layer and after addition layer',\
            'Dropout layer after first Dense layer and after addition layer removing second last dense layer'],\
           prop={'size':20})
plt.show

In [ ]:
model0.save_weights('model_weights_hidden1024/modeltrain.h5')

In [ ]:
model0.load_weights('model_weights_hidden1024/modeltrain.h5')

In [ ]:
images = 'Images/Images/'

In [ ]:
with open('enc_testing.pkl', 'rb') as enc_pkl:
    enc_test = load(enc_pkl)

In [ ]:
pred = []
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model0.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:]
    final = ' '.join(final)
    pred.append(final)
    return final

In [ ]:
pred1 = {}
for i in range(len(enc_test.keys())):
    pic = list(enc_test.keys())[i]
    image1 = enc_test[pic].reshape((1,2048))
    x=plt.imread(images+pic)
    plt.imshow(x)
    plt.show()
    pred1[pic] = []
    pred1[pic].append(greedySearch(image1).split())
    print("Generated Caption:",pic,greedySearch(image1))


In [ ]:
try_img = test_img[13]
Image =open(try_img)


In [ ]:
image2 = trial_vector.reshape(1,2048)
y = plt.imread('pic_screen.png')
plt.imshow(y)
plt.show()
greedySearch(image2)

In [ ]:
for val in train_features.values():
    print(val.shape)

In [ ]:
pred_trail=pred1

In [ ]:
pred_trial_temp = {}

for key in pred_trail.keys():
    a = key.split('.')[0]
    pred_trial_temp[a] = pred_trail[key]

In [ ]:
ref = {}
for keys in pred_trial_temp.keys():
    if keys in description.keys():
        reff = [d.split() for d in description[keys]]
        ref[keys] = description[keys][0:5]
        

In [ ]:
pred_cap = []

for k,v in pred_trial_temp.items():
    pred_cap.append(v)

In [ ]:
pred_cap = [item for i in pred_cap for item in i]

In [ ]:
references = []

for k,v in ref.items():
    references.append(v)

In [ ]:
references = np.array(references)
references = references[:,:, None ]


In [ ]:
for i in references:
    for j in i:
        print (j[0])

In [ ]:
def bleu_score(pred, ref):
    return nltk.translate.bleu_score.corpus_bleu(ref, pred)

In [ ]:
for index, i in enumerate(references):
    print(i)
    print(pred_cap[index])
    print(sentence_bleu(i, pred_cap[index], weights = (1,0,0,0)))
    print(sentence_bleu(i, pred_cap[index], weights = (0.5,0.5,0,0)))
    print(sentence_bleu(i, pred_cap[index], weights = (0.33,0.33,0.33,0)))
    print(sentence_bleu(i, pred_cap[index], weights = (0.25,0.25,0.25,0.25)))

In [ ]:
anno = references[0:1001]
anno

In [ ]:
cap1 = []
for i in range(anno.shape[0]):
    c = []
    for j in range(anno.shape[1]):
        line = anno[i][j]
        linearr = line[0].split(' ')
        c.append(linearr)
    cap1.append(c)
print(cap1)
                   

In [ ]:
p = pred_cap[0:1001]

In [ ]:
q=[]

for i in p:
    print(i)
    q.append(i[0:11])
    #q.append(i.split(' '))

# XCEPTION

In [ ]:
model3 = Xception(weights = 'imagenet')

In [ ]:
xception = Model(model3.input, model3.layers[-2].output)

In [ ]:
def vector2(img):
    img = preprocess(img)
    featvec = xception.predict(img)
    featvec = np.reshape(featvec, featvec.shape[1])
    return featvec

In [ ]:
# encoding all train images

check = time()
enc_train2 = {}

for img in train_img:
    enc_train2[img[len(images):]] = vector2(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_trainimg2.pkl', 'wb') as enc_pkl2:
    pickle.dump(enc_train2, enc_pkl2)

In [ ]:
check = time()
enc_val2 = {}

for img in val_img:
    enc_val2[img[len(images):]] = vector2(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_valimg2.pkl', 'wb') as enc_valpkl2:
    pickle.dump(enc_val2, enc_valpkl2)
val_features2 = load(open('enc_valimg2.pkl', 'rb'))

In [ ]:
# Encoding all test images
check = time()
enc_test2 = dict()

for img in test_img:
    enc_test2[img[len(images):]] = vector2(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving test feature

with open('enc_testing2.pkl', 'wb') as enc_pickle2:
    pickle.dump(enc_test2, enc_pickle2)

In [ ]:
train_features2 = load(open('enc_trainimg2.pkl','rb'))

In [ ]:
# LSTM

inp1 = Input(shape = (2048,))
ffee1 = Dropout(0.5)(inp1)
ffee2 = Dense(512, activation = 'relu')(ffee1)

inp2 = Input(shape = (max_length,))
ssee1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inp2)
ssee2 = Dropout(0.5)(ssee1)
ssee3 = LSTM(512)(ssee2)
dcd1 = add([ffee2, ssee3])
dcd2 = Dense(512, activation = 'relu')(dcd1)
opt = Dense(vocab_size, activation = 'softmax')(dcd2)
modelxception = Model(inputs= [inp1, inp2], output= opt)

In [ ]:
modelxception.layers[2].set_weights([embedding_matrix])
modelxception.layers[2].trainable = False

In [ ]:
start_lr = 0.001
def schedule(epoch):
    return start_lr

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(schedule)

In [ ]:
modelxception.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
epochs = 20
number_photo_per_batch = 32
steps = len(tdict)// number_photo_per_batch

In [ ]:
#Epochs with dropout layer changed from 2nd to 3rd and 6th to 9th with no dense layer removed.
loss0_xception=[]
accuracy0_xception= []
for j in range(epochs):
    generator = data_generator(tdict, train_features2, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features2, wordtoix, max_length, number_photo_per_batch)
    history=modelxception.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_xception.append(history.history['val_loss'])
    accuracy0_xception.append(history.history['val_accuracy'])
    modelxception.save('model_weights2/model' + str(j) +'.h5' )
round(modelxception.optimizer.lr.numpy(),10)

In [ ]:
for j in range(epochs):
    generator2 = data_generator(tdict, train_features2, wordtoix, max_length, number_photo_per_batch)
    model1.fit_generator(generator2, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model1.save('model_weights2/model' + str(j) +'.h5' )

In [ ]:
for j in range(epochs):
    generator2 = data_generator(tdict, train_features2, wordtoix, max_length, number_photo_per_batch)
    model1.fit_generator(generator2, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model1.save('model_weights2/model' + str(j) +'.h5' )

In [ ]:
'''for j in range(epochs):
    generator2 = data_generator(tdict, train_features2, wordtoix, max_length, number_photo_per_batch)
    model.fit_generator(generator2, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model.save('model_weights2/model' + str(j) +'.h5' )'''

In [ ]:
modelxception.save_weights('model_weights2/modeltrain.h5')

In [ ]:
modelxception.load_weights('model_weights2/modeltrain.h5')

In [ ]:
with open('enc_testing2.pkl', 'rb') as enc_pkl:
    enc_test2 = load(enc_pkl)

In [ ]:
pred3 = []
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        photo = photo.reshape(1,2048)
        sequence = pad_sequences([sequence], maxlen=max_length).reshape(1, max_length)
        yhat = modelxception.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:]
    final = ' '.join(final)
    pred3.append(final)
    return final

In [ ]:
pred4 = {}
for i in range(len(enc_test2.keys())):
    pic = list(enc_test2.keys())[i]
    image1 = enc_test2[pic].reshape((1,2048))
    x=plt.imread(images+pic) 
    plt.imshow(x)
    plt.show()
    pred4[pic] = []
    pred4[pic].append(greedySearch(image1).split())
    print("Generated Caption:",greedySearch(image1))

In [ ]:
pred_trail2=pred4 

In [ ]:
pred_trail_temp2 = {}
for keys in pred_trail2:
    key = keys.split('.')[0]
    pred_trail_temp2[key] = pred_trail2[keys]

In [ ]:
ref2 = {}
for keys in pred_trail_temp2.keys():
    if keys in description.keys():
        reff2 = [d.split() for d in description[keys]]
        ref2[keys] = description[keys][0:5]

In [ ]:
references2 = []

for k,v in ref2.items():
    references2.append(v)

In [ ]:
references2

In [ ]:
references2 = np.array(references2)
references2 = references2[:,:, None ]

In [ ]:
references2

In [ ]:
cap2 = []

for i in range(references2.shape[0]):
    c = []
    for j in range(references2.shape[1]):
        line = references2[i][j]
        linearr = line[0].split(' ')
        c.append(linearr)
    cap2.append(c)
print(cap2)

In [ ]:
predtt = pred3

In [ ]:
predtt = set(predtt)

In [ ]:
lst2=[]
for v in pred4.values():
    lst2.append(v)

In [ ]:
lst2=[items for i in lst2 for items in i]

# INCEPTION RESNET V2

In [ ]:
model4 = InceptionResNetV2(weights = 'imagenet')

In [ ]:
inceptionresnet = Model(model4.input, model4.layers[-2].output)

In [ ]:
model4.summary()

In [ ]:
def vector3(img):
    img = preprocess(img)
    featvec = inceptionresnet.predict(img)
    featvec = np.reshape(featvec, featvec.shape[1])
    return featvec

In [ ]:
# encoding all train images

check = time()
enc_train3 = {}

for img in train_img:
    enc_train3[img[len(images):]] = vector3(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_trainimg3.pkl', 'wb') as enc_pkl3:
    pickle.dump(enc_train3, enc_pkl3)

In [ ]:
check = time()
enc_val3 = {}

for img in val_img:
    enc_val3[img[len(images):]] = vector3(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_valimg3.pkl', 'wb') as enc_valpkl3:
    pickle.dump(enc_val3, enc_valpkl3)
val_features3 = load(open('enc_valimg3.pkl', 'rb'))

In [ ]:
# Encoding all test images
check = time()
enc_test3 = dict()

for img in test_img:
    enc_test3[img[len(images):]] = vector3(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving test feature

with open('enc_testing3.pkl', 'wb') as enc_pickle3:
    pickle.dump(enc_test3, enc_pickle3)

In [ ]:
train_features3 = load(open('enc_trainimg3.pkl','rb'))

In [ ]:
for val in train_features3.values():
    print(val.shape)

In [ ]:
# LSTM

input1 = Input(shape = (1536,))
fee1 = Dropout(0.5)(input1)
fee2 = Dense(512, activation = 'relu')(fee1)

input2 = Input(shape = (max_length,))
see1 = Embedding(vocab_size, emb_dim, mask_zero = True)(input2)
see2 = Dropout(0.5)(see1)
see3 = LSTM(512)(see2)
decoders1 = add([fee2, see3])
decoders2 = Dense(512, activation = 'relu')(decoders1)
output = Dense(vocab_size, activation = 'softmax')(decoders2)
modelincpres = Model(inputs= [input1, input2], output= output)

In [ ]:
modelincpres.layers[2].set_weights([embedding_matrix])
modelincpres.layers[2].trainable = False

In [ ]:
modelincpres.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
epochs = 20
number_photo_per_batch = 32
steps = len(tdict)// number_photo_per_batch

In [ ]:
loss0_irv2=[]
accuracy0_irv2= []
for j in range(epochs):
    generator = data_generator(tdict, train_features3, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features3, wordtoix, max_length, number_photo_per_batch)
    history=modelincpres.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_irv2.append(history.history['val_loss'])
    accuracy0_irv2.append(history.history['val_accuracy'])
    modelincpres.save('model_weights3/model' + str(j) +'.h5' )

In [ ]:
for j in range(epochs):
    generator3 = data_generator(tdict, train_features3, wordtoix, max_length, number_photo_per_batch)
    model2.fit_generator(generator3, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model2.save('model_weights3/model' + str(j) +'.h5' )

In [ ]:
for j in range(epochs):
    generator3 = data_generator(tdict, train_features3, wordtoix, max_length, number_photo_per_batch)
    model2.fit_generator(generator3, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model2.save('model_weights3/model' + str(j) +'.h5' )

In [ ]:
modelincpres.save_weights('model_weights2/modeltrain.h5')

In [ ]:
modelincpres.load_weights('model_weights2/modeltrain.h5')

In [ ]:
with open('enc_testing3.pkl', 'rb') as enc_pkl:
    enc_test3 = load(enc_pkl)

In [ ]:
pred3 = []
def greedySearch3(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        photo = photo.reshape(1, 1536)
        sequence = pad_sequences([sequence], maxlen=max_length).reshape(1, max_length)
        yhat = modelincpres.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:]
    final = ' '.join(final)
    return final
    pred3.append(final)
    

In [ ]:
pred4 = {}
for i in range(len(enc_test3.keys())):
    pic = list(enc_test3.keys())[i]
    image1 = enc_test3[pic].reshape((1,1536))
for i in range(len(enc_test3.keys())):
    pic = list(enc_test3.keys())[i]
    image1 = enc_test3[pic]
    x=plt.imread(images+pic) 
    plt.imshow(x)
    plt.show()
    pred4[pic] = []
    pred4[pic].append(greedySearch3(image1).split())
    print("Generated Caption:",greedySearch3(image1))

# NASNET LARGE

In [ ]:
nnet = nasnet.NASNetLarge(weights = 'imagenet')

In [ ]:
nnl = Model(nnet.input, nnet.layers[-2].output)

In [ ]:
nnl.summary()

In [ ]:
def preprocess2(img):
    all_images = image.load_img(img, target_size=(331,331))
    x_axis = image.img_to_array(all_images)
    print(x_axis.shape)
    x_axis = np.expand_dims(x_axis, axis = 0)
    print(x_axis.shape)
    x_axis = preprocess_input(x_axis)
    return x_axis

In [ ]:
for f in os.listdir('Images/Images'):
    all_images = image.load_img(f'Images/Images/{f}', target_size=(331,331))
    x_axis = image.img_to_array(all_images)
    print(x_axis.shape)
    x_axis = np.expand_dims(x_axis, axis = 0)
    print(x_axis.shape)
    x_axis = preprocess_input(x_axis)


In [ ]:
def vector4(img):
    img = preprocess2(img)
    featvec = nnl.predict(img)
    featvec = np.reshape(featvec, featvec.shape[1])
    return featvec

In [ ]:
# encoding all train images

check = time()
enc_train4 = {}

for img in train_img:
    enc_train4[img[len(images):]] = vector4(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_trainimg4.pkl', 'wb') as enc_pkl4:
    pickle.dump(enc_train4, enc_pkl4)

In [ ]:
check = time()
enc_val4 = {}

for img in val_img:
    enc_val4[img[len(images):]] = vector4(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving the features
with open('enc_valimg4.pkl', 'wb') as enc_valpkl4:
    pickle.dump(enc_val4, enc_valpkl4)
val_features4 = load(open('enc_valimg4.pkl', 'rb'))

In [ ]:
# Encoding all test images
check = time()
enc_test4 = dict()

for img in test_img:
    enc_test4[img[len(images):]] = vector4(img)
print('time taken in sec', time()-check)

In [ ]:
#Saving test feature

with open('enc_testing4.pkl', 'wb') as enc_pickle4:
    pickle.dump(enc_test4, enc_pickle4)

In [ ]:
train_features4 = load(open('enc_trainimg4.pkl','rb'))

In [ ]:
# LSTM

inputt1 = Input(shape = (4032,))
ffe1 = Dropout(0.5)(inputt1)
ffe2 = Dense(1024, activation = 'relu')(ffe1)

inputt2 = Input(shape = (max_length,))
sse1 = Embedding(vocab_size, emb_dim, mask_zero = True)(inputt2)
sse2 = Dropout(0.5)(sse1)
sse3 = LSTM(1024)(sse2)
decoderss1 = add([ffe2, sse3])
decoderss2 = Dense(1024, activation = 'relu')(decoderss1)
outputt = Dense(vocab_size, activation = 'softmax')(decoderss2)
modelnnl = Model(inputs= [inputt1, inputt2], output= outputt)

In [ ]:
modelnnl.layers[2].set_weights([embedding_matrix])
modelnnl.layers[2].trainable = False

In [ ]:
def scheduler(epoch, lr):
    if epoch<5:
        return lr
    else:
        return lr*tf.math.exp(-0.1)

In [ ]:
callback2 = tf.keras.callbacks.LearningRateScheduler(scheduler)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'auto', min_delta = 0, patience = 1, verbose = 1)

In [ ]:
optimizer0 = keras.optimizers.adagrad(learning_rate=0.1)

In [ ]:
optimizer1 = keras.optimizers.adagrad(learning_rate = 0.001)

In [ ]:
optimizer2 = keras.optimizers.adagrad(learning_rate=0.0001)

In [ ]:
modelnnl.compile(loss = 'categorical_crossentropy', optimizer = optimizer0, metrics=['accuracy'])

In [ ]:
modelnnl.compile(loss = 'categorical_crossentropy', optimizer = optimizer1, metrics=['accuracy'])

In [ ]:
modelnnl.compile(loss = 'categorical_crossentropy', optimizer = optimizer2, metrics=['accuracy'])

In [ ]:
modelnnl.compile(loss = 'categorical_crossentropy', optimizer = 'adagrad', metrics=['accuracy'])

In [ ]:
epochs = 20
number_photo_per_batch = 32
steps = len(tdict)// number_photo_per_batch

In [ ]:
#adagrad
loss0_nnl_adagrad=[]
accuracy0_nnl_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_nnl_adagrad.append(history.history['val_loss'])
    accuracy0_nnl_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with batch 1024 
loss0_nnl_adagrad=[]
accuracy0_nnl_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_nnl_adagrad.append(history.history['val_loss'])
    accuracy0_nnl_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with learning rate 0.1 
loss0_op0_adagrad=[]
accuracy0_op0_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_op0_adagrad.append(history.history['val_loss'])
    accuracy0_op0_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with learning rate 0.001 
loss0_op1_adagrad=[]
accuracy0_op1_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_op1_adagrad.append(history.history['val_loss'])
    accuracy0_op1_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with learning rate 0.001 including callback
loss0_op1c_adagrad=[]
accuracy0_op1c_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1,callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_op1c_adagrad.append(history.history['val_loss'])
    accuracy0_op1c_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with learning rate 0.0001 
loss0_op2_adagrad=[]
accuracy0_op2_adagrad= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, validation_data = generator2, validation_steps = steps )
    loss0_op2_adagrad.append(history.history['val_loss'])
    accuracy0_op2_adagrad.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adagrad with earlystopping
loss0_nnl_adagrades=[]
accuracy0_nnl_adagrades= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks= [callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adagrades.append(history.history['val_loss'])
    accuracy0_nnl_adagrades.append(history.history['val_accuracy'])
    modelnnl.save('model_weights4/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
accuracy0_nnl_adagrades

In [ ]:
#SGD
loss0_nnl_sgd=[]
accuracy0_nnl_sgd= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_sgd.append(history.history['val_loss'])
    accuracy0_nnl_sgd.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlsgd/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#SGD
loss0_nnl_sgdes=[]
accuracy0_nnl_sgdes= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2,earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_sgdes.append(history.history['val_loss'])
    accuracy0_nnl_sgdes.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlsgd/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adam
loss0_nnl_adam=[]
accuracy0_nnl_adam= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adam.append(history.history['val_loss'])
    accuracy0_nnl_adam.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladam/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adam with earlystopping
loss0_nnl_adames=[]
accuracy0_nnl_adames= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adames.append(history.history['val_loss'])
    accuracy0_nnl_adames.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladam/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adadelta
loss0_nnl_adadelta=[]
accuracy0_nnl_adadelta= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adadelta.append(history.history['val_loss'])
    accuracy0_nnl_adadelta.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladadelta/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adadelta with earlystopping
loss0_nnl_adadeltaes=[]
accuracy0_nnl_adadeltaes= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adadeltaes.append(history.history['val_loss'])
    accuracy0_nnl_adadeltaes.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladadelta/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adamx
loss0_nnl_adamax=[]
accuracy0_nnl_adamax= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adamax.append(history.history['val_loss'])
    accuracy0_nnl_adamax.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladamax/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#adamx with earlystopping
loss0_nnl_adamaxes=[]
accuracy0_nnl_adamaxes= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_adamaxes.append(history.history['val_loss'])
    accuracy0_nnl_adamaxes.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnladamax/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#nadam
loss0_nnl_nadam=[]
accuracy0_nnl_nadam= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_nadam.append(history.history['val_loss'])
    accuracy0_nnl_nadam.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlnadam/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#nadam with earlystopping
loss0_nnl_nadames=[]
accuracy0_nnl_nadames= []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_nadames.append(history.history['val_loss'])
    accuracy0_nnl_nadames.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlnadam/model' + str(j) +'.h5' )
print(modelnnl.optimizer.lr.numpy(),10)

In [ ]:
#rmsprop
loss0_nnl_rmsprop =[]
accuracy0_nnl_rmsprop = []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 1, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2], validation_data = generator2, validation_steps = steps )
    loss0_nnl_rmsprop.append(history.history['val_loss'])
    accuracy0_nnl_rmsprop.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlrmsprop/model' + str(j) +'.h5' )
lrr = modelnnl.optimizer.lr

In [ ]:
#rmsprop with earlystopping
loss0_nnl_rmspropes =[]
accuracy0_nnl_rmspropes = []
for j in range(epochs):
    generator = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    generator2 = data_generator(valdict, val_features4, wordtoix, max_length, number_photo_per_batch)
    history=modelnnl.fit_generator(generator, epochs = 20, steps_per_epoch= steps ,verbose = 1, callbacks=[callback2, earlystopping], validation_data = generator2, validation_steps = steps )
    loss0_nnl_rmspropes.append(history.history['val_loss'])
    accuracy0_nnl_rmspropes.append(history.history['val_accuracy'])
    modelnnl.save('model_weights_nnlrmsprop/model' + str(j) +'.h5' )
lrr = modelnnl.optimizer.lr

In [ ]:
for j in range(epochs):
    generator4 = data_generator(tdict, train_features4, wordtoix, max_length, number_photo_per_batch)
    model3.fit_generator(generator4, epochs = 1, steps_per_epoch= steps ,verbose = 1 )
    model3.save('model_weights4/model' + str(j) +'.h5' )

In [ ]:
loss0_inception = loss0_drop
accuracy0_inception = accuracy0_drop

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(accuracy0_inception, linewidth = 5)
plt.plot(accuracy0_xception, linewidth = 5)
plt.plot(accuracy0_irv2, linewidth = 5)
plt.plot(accuracy0_nnl, linewidth = 5)
plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Inception V3', 'Xception', \
            'Inception ResNet V2',\
            'NASNetLarge'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(loss0_inception, linewidth = 5)
plt.plot(loss0_xception, linewidth = 5)
plt.plot(loss0_irv2, linewidth = 5)
plt.plot(loss0_nnl, linewidth = 5)
plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['Inception V3', 'Xception', \
            'Inception ResNet V2',\
            'NASNetLarge'],\
           prop={'size':20})
plt.show

In [ ]:
loss0_nnl_sgd

# Visualization with different optimizers

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(loss0_nnl_sgd, linewidth = 5)
plt.plot(loss0_nnl_adam, linewidth = 5)
plt.plot(loss0_nnl_nadam, linewidth = 5)
plt.plot(loss0_nnl_adamax, linewidth = 5)
plt.plot(loss0_nnl_adadelta, linewidth = 5)
plt.plot(loss0_nnl_rmsprop, linewidth = 5)
plt.plot(loss0_nnl_adagrad, linewidth = 5)
plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['SGD- lr- 0.01', 'ADAM- lr- 0.001', \
            'NADAM -lr- 0.001',\
            'ADAMAX- lr- 0.002', 'ADADELTA- lr- 1.0',\
            'RMSPROP- lr- 0.001', 'Adagrad- lr- 0.01'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(accuracy0_nnl_sgd, linewidth = 5)
plt.plot(accuracy0_nnl_adam, linewidth = 5)
plt.plot(accuracy0_nnl_nadam, linewidth = 5)
plt.plot(accuracy0_nnl_adamax, linewidth = 5)
plt.plot(accuracy0_nnl_adadelta, linewidth = 5)
plt.plot(accuracy0_nnl_rmsprop, linewidth = 5)
plt.plot(accuracy0_nnl_adagrad, linewidth = 5)
plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['SGD- lr- 0.01', 'ADAM- lr- 0.001', \
            'NADAM -lr- 0.001',\
            'ADAMAX- lr- 0.002', 'ADADELTA- lr- 1.0',\
            'RMSPROP- lr- 0.001', 'Adagrad- lr- 0.01'],\
           prop={'size':20})
plt.show

# Experiments with AdaGrad

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(loss0_op0_adagrad, linewidth = 5)
plt.plot(loss0_nnl_adagrad, linewidth = 5)
plt.plot(loss0_op1_adagrad, linewidth = 5)
plt.plot(loss0_op2_adagrad, linewidth = 5)
plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['AdaGrad lr-0.1', 'AdaGrad lr-0.01' ,'AdaGrad lr-0.001', \
            'AdaGrad lr-0.0001'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))
plt.plot(accuracy0_op0_adagrad, linewidth = 5)
plt.plot(accuracy0_nnl_adagrad, linewidth = 5)
plt.plot(accuracy0_op1_adagrad, linewidth = 5)
plt.plot(accuracy0_op2_adagrad, linewidth = 5)
plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['AdaGrad lr-0.1', 'AdaGrad lr-0.01' ,'AdaGrad lr-0.001', \
            'AdaGrad lr-0.0001'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))

plt.plot(loss0_nnl_adagrad, linewidth = 5)
plt.plot(loss0_op1c_adagrad, linewidth = 5)

plt.plot(loss0_op1_adagrad, linewidth = 5)

plt.ylabel('Validation Loss', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['AdaGrad','AdaGrad lr-0.001 with learning rate scheduler', 'AdaGrad lr-0.001'],\
           prop={'size':20})
plt.show

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
plt.figure(figsize = (25,15))

plt.plot(accuracy0_nnl_adagrad, linewidth = 5)
plt.plot(accuracy0_op1c_adagrad, linewidth = 5)

plt.plot(accuracy0_op1_adagrad, linewidth = 5)

plt.ylabel('Validation Accuracy', fontsize = 25)
plt.xlabel('Epochs', fontsize=25)
plt.legend(['AdaGrad','AdaGrad lr-0.001 with learning rate scheduler', 'AdaGrad lr-0.001'],\
           prop={'size':20})
plt.show

In [ ]:
modelnnl.save_weights('model_weights4/modeltrain.h5')

In [ ]:
modelnnl.load_weights('model_weights4/modeltrain.h5')

In [ ]:
with open('enc_testing4.pkl', 'rb') as enc_pickle4:
    enc_test4 = load(enc_pickle4)

In [ ]:
pred5 = []
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        photo = photo.reshape(1,4032)
        sequence = pad_sequences([sequence], maxlen=max_length).reshape(1, max_length)
        yhat = modelnnl.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:]
    final = ' '.join(final)
    pred5.append(final)
    return final

In [ ]:
pred4 = {}
for i in range(len(enc_test4.keys())):
    pic = list(enc_test4.keys())[i]
    image1 = enc_test4[pic].reshape((1,4032))
    x=plt.imread(images+pic) 
    plt.imshow(x)
    plt.show()
    pred4[pic] = []
    pred4[pic].append(greedySearch(image1).split())
    print("Generated Caption:",greedySearch(image1))

In [ ]:
pred_trial4 = pred4

In [ ]:
pred_trial_temp4 = {}
for keys in pred_trial4:
    key = keys.split('.')[0]
    pred_trial_temp4[key] = pred_trial4[keys]

In [ ]:
ref4 = {}
for keys in pred_trial_temp4.keys():
    if keys in description.keys():
        reff = [d.split() for d in description[keys]]
        ref4[keys] = description[keys][0:5]
        

In [ ]:
pred_cap4 = []

for k,v in pred_trial_temp4.items():
    pred_cap4.append(v)

In [ ]:
pred_cap4 = [item for i in pred_cap4 for item in i]

In [ ]:

references4 = []

for k,v in ref4.items():
    references4.append(v)

In [ ]:
references4 = np.array(references4)
references4 = references4[:,:, None ]


In [ ]:
anno = references4[0:1001]
anno

In [ ]:
cap4 = []
for i in range(anno.shape[0]):
    c = []
    for j in range(anno.shape[1]):
        line = anno[i][j]
        linearr = line[0].split(' ')
        c.append(linearr)
    cap4.append(c)
print(cap4)
           


In [ ]:
cap1024 = []
for i in range(anno.shape[0]):
    c = []
    for j in range(anno.shape[1]):
        line = anno[i][j]
        linearr = line[0].split(' ')
        c.append(linearr)
    cap1024.append(c)
print(cap1024)

In [ ]:
pred_captions = pred_cap4[0:1001]

In [ ]:
captions=[]

for i in p:
    captions.append(i[0:7])


In [ ]:
# BLEU Score batch hidden unit size 512
b1512#(corpus_bleu(cap4, q, weights=(1,0,0,0)))
b2512#(corpus_bleu(cap4, q, weights=(0.5,0.5,0,0)))
b3512#(corpus_bleu(cap4, q, weights=(0.3,0.3,0.3,0)))
b4512#(corpus_bleu(cap4, q, weights=(0.25,0.25,0.25,0.25)))

In [ ]:
# BLEU Score batch hidden unit size 1024
b11024=(corpus_bleu(cap4, q, weights=(1,0,0,0)))
b21024=(corpus_bleu(cap4, q, weights=(0.5,0.5,0,0)))
b31024=(corpus_bleu(cap4, q, weights=(0.3,0.3,0.3,0)))
b41024=(corpus_bleu(cap4, q, weights=(0.25,0.25,0.25,0.25)))